# Module - Preprocessing

In [13]:
# Import to be able to import python package from src
import sys
sys.path.insert(0, '../src')

In [14]:
import pandas as pd
import numpy as np
from darts.datasets import EnergyDataset

In [15]:
import ontime as on

---
## Load data

In [16]:
ts = EnergyDataset().load()
ts = ts.astype(np.float32)

---

## Tensorflow Preprocessing

In [17]:
from ontime.module import preprocessing

### Time series data

In [56]:
ts_t = preprocessing.common.normalize(ts)

train, test = preprocessing.common.train_test_split(ts_t, train_split=0.8)

train_list = preprocessing.common.split_by_length(train, 6)
test_list = preprocessing.common.split_by_length(test, 6)

X_train, y_train = preprocessing.common.split_inputs_from_targets(train_list, 4, 2)
X_test, y_test = preprocessing.common.split_inputs_from_targets(test_list, 4, 2)

X_train = preprocessing.common.timeseries_list_to_numpy(X_train)
y_train = preprocessing.common.timeseries_list_to_numpy(y_train)
X_test = preprocessing.common.timeseries_list_to_numpy(X_test)
y_test = preprocessing.common.timeseries_list_to_numpy(y_test)

/Users/fred.montet/Library/Caches/pypoetry/virtualenvs/ontime-FpQu8-YN-py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/Users/fred.montet/Library/Caches/pypoetry/virtualenvs/ontime-FpQu8-YN-py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [61]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4675, 4, 28)
(4675, 2, 28)
(1168, 4, 28)
(1168, 2, 28)


## MNIST for testing

In [62]:
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

In [63]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


## Modelling

In [43]:
import tensorflow as tf

In [58]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(4, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2)
])

In [59]:
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)

absl       WARNING At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.


In [60]:
model.fit(
    X_train,
    y_train,
    epochs=4
)

Epoch 1/4


ValueError: in user code:

    File "/Users/fred.montet/Library/Caches/pypoetry/virtualenvs/ontime-FpQu8-YN-py3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/Users/fred.montet/Library/Caches/pypoetry/virtualenvs/ontime-FpQu8-YN-py3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/fred.montet/Library/Caches/pypoetry/virtualenvs/ontime-FpQu8-YN-py3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/Users/fred.montet/Library/Caches/pypoetry/virtualenvs/ontime-FpQu8-YN-py3.10/lib/python3.10/site-packages/keras/src/engine/training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/fred.montet/Library/Caches/p

In [55]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 353us/step - loss: 0.9698 - mean_absolute_error: 0.6702


[0.9698350429534912, 0.670210063457489]